In [1]:
!rm -rf /kaggel/working/*

In [2]:
!git clone https://github.com/taduyhai3k/machine_trans_ver2

Cloning into 'machine_trans_ver2'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 73 (delta 18), reused 72 (delta 17), pack-reused 0 (from 0)
Unpacking objects: 100% (73/73), 21.08 MiB | 8.59 MiB/s, done.


In [3]:
import os, sys
# lấy ra đường dẫn đến thư mục modules ở trong projetc hiện hành
lib_path = os.path.abspath(os.path.join('machine_trans_ver2'))
# thêm thư mục cần load vào trong hệ thống
sys.path.append(lib_path)
import argparse
from machine_trans_ver2 import utils,  MyData, MyTrans

In [4]:
import torch
import pandas as pd
a = torch.load("/kaggle/input/para/pytorch/default/1/bestmodel_parallel.pth", map_location = 'cuda')
torch.save(a, "machine_trans_ver2/checkpoint/bestmodel_parallel.pth")
b = pd.read_csv("/kaggle/input/d/hai3kk/metric-v2e/metric_V2E.csv")
b.to_csv("machine_trans_ver2/metric/metric_E2V.csv")



In [5]:
import argparse 

parse = argparse.ArgumentParser(description="machine translation using transformer")
parse.add_argument("--path_train", type=str, help="path to data train", default='data/train1.json')
parse.add_argument("--path_valid", type=str, help="path to data valid", default='data/valid1.json')
parse.add_argument("--path_test", type=str, help="path to data test", default='data/test1.json')
parse.add_argument("--in_lang", type=str, help="input language", default='E')
parse.add_argument("--out_lang", type=str, help="output language", default='V')
parse.add_argument("--vocab_E", type=str, help="path Vocabulary of E", default='vocab/vocabE1.json')
parse.add_argument("--vocab_V", type=str, help="path Vocabulary of V", default='vocab/vocabV1.json')
parse.add_argument("--dmodel", type=int, help="Dimension of model", default=128)
parse.add_argument("--dembed", type=int, help="Dimension of embedding", default=128)
parse.add_argument("--d_ff", type=int, help="Dimension of feed-forward layer", default=256)
parse.add_argument("--head", type=int, help="Number of attention heads", default=8)
parse.add_argument("--active", type=str, help="Type of activation function", default="relu")
parse.add_argument("--layer", type=int, help="Number of layers", default=2)
parse.add_argument("--dropout", type=float, help="Dropout rate", default=0.1)
parse.add_argument("--eps", type=float, help="Epsilon value", default=1e-5)
parse.add_argument("--epoch", type= int, default= 12, help= "epoch")
parse.add_argument("--batch_size", type= int, default= 128, help = "batch size in training and testing")
parse.add_argument("--result_path", type= str, default= "checkpoint/bestmodel_E2V.pth", help = "file path to result model")
parse.add_argument("--metric_path", type= str, default= "metric/metric_V2E.csv", help = "file path to result metric")
parse.add_argument("--folder", type = str, default = "machine_trans_ver2", help = "folder have data, checkpoint, metric")
args, unknown = parse.parse_known_args()

path_train = os.path.join(args.folder, args.path_train)
path_test = os.path.join(args.folder, args.path_test)
path_valid = os.path.join(args.folder, args.path_valid)
vocab_E = os.path.join(args.folder, args.vocab_E)
vocab_V = os.path.join(args.folder, args.vocab_V)
result_path = os.path.join(args.folder, args.result_path)
metric_path = os.path.join(args.folder, args.metric_path)
data_train = MyData.EV_Data(path_train, inp = args.in_lang, out = args.out_lang, E_vocab_path= vocab_E, V_vocab_path= vocab_V)
model = MyTrans.TransformerParallel(input_vocab_size= len(data_train.inp_vocab), output_vocab_size = len(data_train.out_vocab), dmodel = args.dmodel, dembed = args.dembed,
                        d_ff= args.d_ff, head = args.head, active= args.active, layer= args.layer, dropout= args.dropout, eps = args.eps)
# model = MyTrans.Transformer(input_vocab_size= len(data_train.inp_vocab), output_vocab_size = len(data_train.out_vocab), dmodel = args.dmodel, dembed = args.dembed,
#                         d_ff= args.d_ff, head = args.head, active= args.active, layer= args.layer, dropout= args.dropout, eps = args.eps)
data_train = MyData.DataLoader(data_train, batch_size= args.batch_size, shuffle= False)
if args.path_valid is not None:
    data_valid = MyData.EV_Data(path_valid, inp = args.in_lang, out = args.out_lang, E_vocab_path= vocab_E, V_vocab_path= vocab_V)
    data_valid = MyData.DataLoader(data_valid, batch_size= args.batch_size, shuffle= False)
    data_test = MyData.EV_Data(path_test, inp = args.in_lang, out = args.out_lang, E_vocab_path= vocab_E, V_vocab_path= vocab_V)
    data_test = MyData.DataLoader(data_test, batch_size= args.batch_size, shuffle= False)
else:
    data_valid = None
    data_test = None
# optimizer = utils.optim.Adam(model.parameters(), lr = 1.0, betas= (0.9, 0.98), eps= 1e-9)     
# utils.train(model, optimizer, args.epoch, data_train, data_valid, data_test,result_path, metric_path)
model.load_state_dict(a['model'])
    



<All keys matched successfully>

In [6]:
total_params = sum(p.numel() for p in model.parameters())
print(total_params)

9650688


In [7]:
import json
model.load_state_dict(a['model'])
with open(vocab_E, "r", encoding='utf-8') as file:
    vocabE = json.load(file)  
with open(vocab_V, "r", encoding='utf-8') as file:
    vocabV = json.load(file)  
    
E_tokenizer = MyData.MyTokenizer(vocabE)
V_tokenizer = MyData.MyTokenizer(vocabV)      

In [ ]:
from tqdm import tqdm
def predict(model, data, inp_tokennizer, out_tokenizer, max_lenth = 300):
    #translate one sentences
    inp_encode = data.reshape(1,-1).to('cuda')
    out_encode = out_tokenizer.encode('<start>')[1].reshape(1, -1).to('cuda')
    for i in range(max_lenth):
        pred = model(inp_encode, out_encode)
        preds = pred[:, -1, :].reshape(1, -1)
        predict_id = torch.argmax(preds, dim = -1, keepdim= True).type(torch.int64)
        out_encode = torch.cat([out_encode, predict_id], dim = -1)
        if out_tokenizer.encode('<end>')[1] == predict_id:
            return out_tokenizer.decode(out_encode.reshape(-1))
    return out_tokenizer.decode(out_encode.reshape(-1))     

def predict_parallel(model, data, inp_tokennizer, out_tokenizer, inp = 'E',max_lenth = 300):
    #translate one sentences
    model.eval()
    inp_encode = inp_tokennizer.encode(data).reshape(1, -1)
    out_encode = out_tokenizer.encode('<start>')[1].reshape(1, -1)
    with torch.no_grad():
        for i in range(max_lenth):
            pred = model(inp_encode, out_encode, inp)
            preds = pred[:, -1, :].reshape(1, -1)
            predict_id = torch.argmax(preds, dim = -1, keepdim= True).type(torch.int64)
            out_encode = torch.cat([out_encode, predict_id], dim = -1)
            if out_tokenizer.encode('<end>')[1] == predict_id:
                return out_tokenizer.decode(out_encode.reshape(-1))
    return out_tokenizer.decode(out_encode.reshape(-1))     
candidate = []
reference = []
model.eval()
datas = [data_train, data_test, data_valid]
with torch.no_grad():
    for data in datas:
        data = tqdm(data, desc='Process', position=0, leave=True) 
        for batch in data:
            inp, target = batch[0].to('cuda'), batch[1].to('cuda')
            tmp_can = model.inference(target, 16,'V',300)
            candidate = candidate + tmp_can
            reference = reference + inp.tolist()
            
            

Process:  50%|████▉     | 104/209 [01:03<01:02,  1.67it/s]

In [11]:
length = 0
for i in range(len(candidate)):
    length += len(candidate[i])

print(length * 1.0 /len(candidate) )    

13.937399024480312


In [37]:
a = '","'.join(str(x) for x in reference[0])

In [38]:
a

'1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","0","

In [12]:
import nltk
from nltk.translate.bleu_score import sentence_bleu

# Tính chỉ số BLEU cho một cặp câu
def calculate_bleu(reference, candidate):
    """
    Tính điểm BLEU giữa câu tham chiếu và câu dự đoán.
    
    :param reference: List chứa các câu tham chiếu, mỗi câu là một list các từ (tokens).
                      VD: [["this", "is", "a", "test"], ["this", "is", "test"]]
    :param candidate: List chứa câu dự đoán (tokens).
                      VD: ["this", "is", "a", "test"]
    :return: Điểm BLEU.
    """
    candidate = '","'.join(str(x) for x in candidate.tolist())
    reference = '","'.join(str(x) for x in reference if x != 0) 
    bleu_score = sentence_bleu(reference, candidate)
    return bleu_score

# Ví dụ sử dụng
point = 0
for i in tqdm(range(len(candidate))):
    point += calculate_bleu(reference[i], candidate[i])
print(f"BLEU Score: {point / len(candidate)}")

100%|██████████| 389946/389946 [54:00<00:00, 120.33it/s] 

BLEU Score: 0.6504610979909451


In [20]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
from joblib import Parallel, delayed

# Tính chỉ số BLEU cho một cặp câu
def calculate_bleu(reference, candidate):
    """
    Tính điểm BLEU giữa câu tham chiếu và câu dự đoán.
    
    :param reference: List chứa các câu tham chiếu, mỗi câu là một list các từ (tokens).
                      VD: [["this", "is", "a", "test"], ["this", "is", "test"]]
    :param candidate: List chứa câu dự đoán (tokens).
                      VD: ["this", "is", "a", "test"]
    :return: Điểm BLEU.
    """
    candidate = '","'.join(str(x) for x in candidate.tolist())
    reference = '","'.join(str(x) for x in reference if x != 0) 
    bleu_score = sentence_bleu(reference, candidate)
    return bleu_score

# Hàm xử lý song song trên nhiều cặp câu
def parallel_bleu(reference_list, candidate_list, n_jobs=-1):
    """
    Tính BLEU song song cho nhiều cặp câu.
    
    :param reference_list: List các câu tham chiếu (tokens).
    :param candidate_list: List các câu dự đoán (tokens).
    :param n_jobs: Số lượng CPU core để sử dụng (mặc định là tất cả).
    :return: List các điểm BLEU.
    """
    bleu_scores = Parallel(n_jobs=n_jobs)(delayed(calculate_bleu)(ref, cand) for ref, cand in zip(reference_list, candidate_list))
    return bleu_scores

a = parallel_bleu(reference, candidate)

/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/opt/conda/lib/python3.10/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


KeyboardInterrupt: 

In [7]:
from IPython.display import FileLink
FileLink(r'machine_translation_viet_eng/metric/metric_E2V.csv')

/kaggle/working/machine_translation_viet_eng/metric/metric_E2V.csv

In [ ]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
batch_size = 64
vocab_E = os.path.join("machine_translation_viet_eng","vocab/vocabE1.json")
vocab_V = os.path.join("machine_translation_viet_eng", "vocab/vocabV1.json")

data_train = MyData.EV_Data("machine_translation_viet_eng/data/train1.json", inp = "E", out = "V", E_vocab_path= vocab_E, V_vocab_path= vocab_V)
data_train = MyData.DataLoader(data_train, batch_size= batch_size, shuffle= False)
data_valid = MyData.EV_Data("machine_translation_viet_eng/data/valid1.json", inp = "E", out = "V", E_vocab_path= vocab_E, V_vocab_path= vocab_V)
data_valid = MyData.DataLoader(data_valid, batch_size= batch_size, shuffle= False)
data_test = MyData.EV_Data("machine_translation_viet_eng/data/test1.json", inp = "E", out = "V", E_vocab_path= vocab_E, V_vocab_path= vocab_V)
data_test = MyData.DataLoader(data_test, batch_size= batch_size, shuffle= False)

d
smoothing_func = SmoothingFunction().method1
bleu_score = sentence_bleu([reference], candidate, smoothing_function=smoothing_func)

In [ ]:
a = (torch.gather(tmp.softmax(dim = -1)[:, :-1, :], dim = -1, index = y.unsqueeze(dim = -1)[:, 1:,:]))

In [ ]:
a[a< 1e-40] = 1e-40

In [ ]:
a.shape

In [ ]:
utils.SparseCrossEntropy(y, tmp)